## SVM + TDIDF

In [120]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [168]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
import seaborn as sns
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix

In [169]:
data = pd.read_csv('/content/drive/MyDrive/dataset_ner_tsv/merged_file.tsv', sep='\t')

In [170]:
data

,text,label
0,COVID-19,S-Disease
1,COVID-19,S-Disease
2,COVID-19,S-Disease
3,malaria,S-Disease
4,HIV,S-Disease
...,...,...
113149,do,O
113150,Estado,O
113151,do,O
113152,Amazonas,O


In [171]:
data.isnull().sum()

text     1
label    0
dtype: int64

In [172]:
data.dropna(inplace=True)

In [173]:
data.isnull().sum()

text     0
label    0
dtype: int64

In [175]:
#data.duplicated().sum()
#data.drop_duplicates(inplace=True)

In [176]:
data

,text,label
0,COVID-19,S-Disease
1,COVID-19,S-Disease
2,COVID-19,S-Disease
3,malaria,S-Disease
4,HIV,S-Disease
...,...,...
113149,do,O
113150,Estado,O
113151,do,O
113152,Amazonas,O


In [177]:
print(data['label'].value_counts())

O                          105303
S-Disease                    1182
B-Disease                    1016
B-Organism                    961
I-Organism                    536
S-Chemical_Substance          496
S-Protien                     463
S-Organism                    402
B-Chemical_Substance          351
B-Protien                     302
I-Disease                     297
B-Medication                  229
S-Medication                  171
S-Anatomical_Substances       151
I-Protien                     143
B-Anatomical_Substances       143
I-Chemical_Substance          137
I-Anatomical_Substances       122
I-Medication                  104
S-Gene                        101
E-Disease                      83
E-Protien                      80
E-Organism                     75
S_Anatomical_Substances        72
B-Gene                         60
E-Medication                   46
E-Anatomical_Substances        42
E-Chemical_Substance           41
I-Gene                         27
E-Gene        

In [178]:
X= data['text']
y =data['label']

In [179]:
X

0         COVID-19
1         COVID-19
2         COVID-19
3          malaria
4              HIV
            ...   
113149          do
113150      Estado
113151          do
113152    Amazonas
113153      FAPEAM
Name: text, Length: 113153, dtype: object

In [150]:
y

0         S-Disease
3         S-Disease
4         S-Disease
5         B-Disease
6         I-Disease
            ...    
113140            O
113144            O
113146            O
113148            O
113153            O
Name: label, Length: 15394, dtype: object

In [180]:
print(len(set(X)))

14376


In [181]:
print(len(set(y)))

30


In [182]:
from collections import Counter
print(Counter(y))

Counter({'O': 105303, 'S-Disease': 1182, 'B-Disease': 1016, 'B-Organism': 961, 'I-Organism': 536, 'S-Chemical_Substance': 496, 'S-Protien': 463, 'S-Organism': 402, 'B-Chemical_Substance': 351, 'B-Protien': 302, 'I-Disease': 297, 'B-Medication': 229, 'S-Medication': 171, 'S-Anatomical_Substances': 151, 'I-Protien': 143, 'B-Anatomical_Substances': 143, 'I-Chemical_Substance': 137, 'I-Anatomical_Substances': 122, 'I-Medication': 104, 'S-Gene': 101, 'E-Disease': 83, 'E-Protien': 80, 'E-Organism': 75, 'S_Anatomical_Substances': 72, 'B-Gene': 60, 'E-Medication': 46, 'E-Anatomical_Substances': 42, 'E-Chemical_Substance': 41, 'I-Gene': 27, 'E-Gene': 17})


In [69]:
#sns.countplot(data['label'])

In [ ]:
#minority_classes = ['B-Organism', 'B-Disease', 'S-Chemical_Substance', 'S-Protien', 'S-Disease', 'B-Chemical_Substance', 'B-Protien', 'I-Organism', 'S-Organism', 'I-Disease', 'B-Medication', 'I-Protien', 'I-Chemical_Substance', 'S-Medication', 'B-Anatomical_Substances', 'I-Anatomical_Substances', 'I-Medication', 'S-Gene', 'E-Protien', 'S-Anatomical_Substances', 'E-Organism', 'S_Anatomical_Substances', 'E-Disease', 'B-Gene', 'E-Chemical_Substance', 'E-Medication', 'E-Anatomical_Substances', 'I-Gene', 'E-Gene']
#df_minority = data[data['label'].isin(minority_classes)]
#df_majority = data[~data['label'].isin(minority_classes)]
#from sklearn.utils import resample
#randomly undersample the majority class to match the size of the minority class
#majority_class_size = len(df_minority)
#df_majority_downsampled = resample(df_majority,
                                   #replace=False,      # sample without replacement
                                   #n_samples=majority_class_size,  # match minority class size
                                   #random_state=123)   # set random state for reproducibility
#concatenate the minority class dataframe with the downsampled majority class dataframe
#df_downsampled = pd.concat([df_majority_downsampled, df_minority])

In [139]:
#print(Counter(df_downsampled['label']))

In [61]:
#import imblearn
#from imblearn.under_sampling import RandomUnderSampler
#rus = RandomUnderSampler(random_state=42)

In [65]:
#X_rus, y_rus = rus.fit_resample(X, y)

In [108]:
#X= df_downsampled['text']
#y =df_downsampled['label']

In [185]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=0)

In [186]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Create TfidfVectorizer to convert texts to vectors
vectorizer = TfidfVectorizer()

In [187]:
# Convert train and test texts to vectors
X_train_vectors = vectorizer.fit_transform(X_train)
X_test_vectors = vectorizer.transform(X_test)

In [188]:
# Encoding the classes in numerical values
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.fit_transform(y_test)

In [189]:
unique_elements = set(y_train)
num_unique_elements = len(unique_elements)

In [190]:
num_unique_elements

30

In [191]:
# Create SVM classifier
clf = SVC(kernel='rbf', C=1, decision_function_shape='ovr')

In [192]:
# Train the classifier on the training data
clf.fit(X_train_vectors, y_train)

SVC(C=1)

In [193]:
y_pred = clf.predict(X_test_vectors)

In [194]:
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

print("Accuracy:", accuracy)

Accuracy: 0.9474614466881711


In [197]:
#testing on new text
new="""Malaria is an acute febrile illness caused by Plasmodium parasites, which are spread to people through the bites of infected female Anopheles mosquitoes. It is preventable and curable."""
n=new.split()
for i in n:
    vector=vectorizer.transform([i])
    pred=clf.predict(vector)
    print(i,encoder.inverse_transform(pred))

Malaria ['S-Disease']
is ['O']
an ['O']
acute ['O']
febrile ['O']
illness ['O']
caused ['O']
by ['O']
Plasmodium ['B-Organism']
parasites, ['O']
which ['O']
are ['O']
spread ['O']
to ['O']
people ['O']
through ['O']
the ['O']
bites ['O']
of ['O']
infected ['O']
female ['O']
Anopheles ['B-Organism']
mosquitoes. ['O']
It ['O']
is ['O']
preventable ['O']
and ['O']
curable. ['O']


## LOGISTIC REGRESSION + TDIDF

In [198]:
from sklearn.linear_model import LogisticRegression

# Create logistic regression classifier
clsf = LogisticRegression(random_state=0, max_iter=1000, multi_class='ovr')

# Train the classifier on the training data
clsf.fit(X_train_vectors, y_train)

# Predict the labels of the test data
y_pred = clsf.predict(X_test_vectors)

In [199]:
y_pred = clsf.predict(X_test_vectors)

In [200]:
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

print("Accuracy:", accuracy)

Accuracy: 0.9386681984887986


In [202]:
new="""Malaria is an acute febrile illness caused by Plasmodium parasites, which are spread to people through the bites of infected female Anopheles mosquitoes. It is preventable and curable."""
n=new.split()
for i in n:
    vector=vectorizer.transform([i])
    pred=clsf.predict(vector)
    print(i,encoder.inverse_transform(pred))

Malaria ['S-Disease']
is ['O']
an ['O']
acute ['O']
febrile ['O']
illness ['O']
caused ['O']
by ['O']
Plasmodium ['B-Organism']
parasites, ['O']
which ['O']
are ['O']
spread ['O']
to ['O']
people ['O']
through ['O']
the ['O']
bites ['O']
of ['O']
infected ['O']
female ['O']
Anopheles ['B-Organism']
mosquitoes. ['O']
It ['O']
is ['O']
preventable ['O']
and ['O']
curable. ['O']


## RANDOM FOREST + TDIDF

In [203]:
from sklearn.ensemble import RandomForestClassifier

# Create Random Forest classifier
cl = RandomForestClassifier(n_estimators=100, random_state=0)

# Train the classifier on the training data
cl.fit(X_train_vectors, y_train)

# Predict the labels of the test data
y_pred = cl.predict(X_test_vectors)

In [204]:
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

print("Accuracy:", accuracy)

Accuracy: 0.9500684901241659


In [207]:
new="""Malaria is an acute febrile illness caused by Plasmodium parasites, which are spread to people through the bites of infected female Anopheles mosquitoes. It is preventable and curable."""
n=new.split()
for i in n:
    vector=vectorizer.transform([i])
    pred=cl.predict(vector)
    print(i,encoder.inverse_transform(pred))

Malaria ['S-Disease']
is ['O']
an ['O']
acute ['O']
febrile ['O']
illness ['O']
caused ['O']
by ['O']
Plasmodium ['B-Organism']
parasites, ['O']
which ['O']
are ['O']
spread ['O']
to ['O']
people ['O']
through ['O']
the ['O']
bites ['O']
of ['O']
infected ['O']
female ['O']
Anopheles ['B-Organism']
mosquitoes. ['O']
It ['O']
is ['O']
preventable ['O']
and ['O']
curable. ['O']


## NAIVE BAYES+TFIDF

In [208]:
# Create Naive Bayes classifier
c = MultinomialNB()

# Train the classifier on the training data
c.fit(X_train_vectors, y_train)

# Predict the labels of the test data
y_pred = c.predict(X_test_vectors)

In [209]:
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

print("Accuracy:", accuracy)

Accuracy: 0.9305819451195263


In [211]:
new=""" P. falciparum is the deadliest malaria parasite and the most prevalent on the African continent. P. vivax is the dominant malaria parasite in most countries outside of sub-Saharan Africa."""
n=new.split()
for i in n:
    vector=vectorizer.transform([i])
    pred=cl.predict(vector)
    print(i,encoder.inverse_transform(pred))

P. ['O']
falciparum ['I-Organism']
is ['O']
the ['O']
deadliest ['O']
malaria ['S-Disease']
parasite ['O']
and ['O']
the ['O']
most ['O']
prevalent ['O']
on ['O']
the ['O']
African ['O']
continent. ['O']
P. ['O']
vivax ['B-Organism']
is ['O']
the ['O']
dominant ['O']
malaria ['S-Disease']
parasite ['O']
in ['O']
most ['O']
countries ['O']
outside ['O']
of ['O']
sub-Saharan ['O']
Africa. ['O']


In [215]:
#confusion matrix
from sklearn.metrics import ConfusionMatrixDisplay
#ConfusionMatrixDisplay.from_predictions(y_test,y_pred)

In [214]:
# results
#from sklearn.metrics import classification_report
#report = classification_report(y_test,y_pred)
#print(report)

#SVM + COUNT VECTORIZER

In [216]:
X= data['text']
y =data['label']

In [219]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=0)

In [233]:
from sklearn.feature_extraction.text import CountVectorizer

# Create a CountVectorizer object
vec = CountVectorizer()

# Fit the vectorizer to the training data and transform the training data into a bag-of-words matrix
X_train_vectors = vec.fit_transform(X_train)

# Transform the test data into a bag-of-words matrix using the same vectorizer
X_test_vectors = vec.transform(X_test)

In [234]:
# Encoding the classes in numerical values
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.fit_transform(y_test)

In [222]:
# Create SVM classifier
clf = SVC(kernel='rbf', C=1, decision_function_shape='ovr')

In [223]:
clf.fit(X_train_vectors, y_train)

SVC(C=1)

In [224]:
y_pred = clf.predict(X_test_vectors)

In [225]:
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

print("Accuracy:", accuracy)

Accuracy: 0.9486103132870841


In [226]:
new="""Malaria is an acute febrile illness caused by Plasmodium parasites, which are spread to people through the bites of infected female Anopheles mosquitoes. It is preventable and curable."""
n=new.split()
for i in n:
    vector=vec.transform([i])
    pred=clf.predict(vector)
    print(i,encoder.inverse_transform(pred))

Malaria ['S-Disease']
is ['O']
an ['O']
acute ['O']
febrile ['O']
illness ['O']
caused ['O']
by ['O']
Plasmodium ['B-Organism']
parasites, ['O']
which ['O']
are ['O']
spread ['O']
to ['O']
people ['O']
through ['O']
the ['O']
bites ['O']
of ['O']
infected ['O']
female ['O']
Anopheles ['B-Organism']
mosquitoes. ['O']
It ['O']
is ['O']
preventable ['O']
and ['O']
curable. ['O']


## LOGISTIC REGRESSION + COUNT VECTORIZER

In [227]:
from sklearn.linear_model import LogisticRegression

# Create logistic regression classifier
clsf = LogisticRegression(random_state=0, max_iter=1000, multi_class='ovr')

# Train the classifier on the training data
clsf.fit(X_train_vectors, y_train)

# Predict the labels of the test data
y_pred = clsf.predict(X_test_vectors)

In [228]:
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

print("Accuracy:", accuracy)

Accuracy: 0.9398170650877116


In [229]:
new="""Malaria is an acute febrile illness caused by Plasmodium parasites, which are spread to people through the bites of infected female Anopheles mosquitoes. It is preventable and curable."""
n=new.split()
for i in n:
    vector=vec.transform([i])
    pred=clsf.predict(vector)
    print(i,encoder.inverse_transform(pred))

Malaria ['S-Disease']
is ['O']
an ['O']
acute ['O']
febrile ['O']
illness ['O']
caused ['O']
by ['O']
Plasmodium ['B-Organism']
parasites, ['O']
which ['O']
are ['O']
spread ['O']
to ['O']
people ['O']
through ['O']
the ['O']
bites ['O']
of ['O']
infected ['O']
female ['O']
Anopheles ['B-Organism']
mosquitoes. ['O']
It ['O']
is ['O']
preventable ['O']
and ['O']
curable. ['O']


## RANDOM FOREST + COUNT VECTORIZER

In [230]:
from sklearn.ensemble import RandomForestClassifier

# Create Random Forest classifier
cl = RandomForestClassifier(n_estimators=100, random_state=0)

# Train the classifier on the training data
cl.fit(X_train_vectors, y_train)

# Predict the labels of the test data
y_pred = cl.predict(X_test_vectors)

In [231]:
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

print("Accuracy:", accuracy)

Accuracy: 0.9500684901241659


In [232]:
new="""Malaria is an acute febrile illness caused by Plasmodium parasites, which are spread to people through the bites of infected female Anopheles mosquitoes. It is preventable and curable."""
n=new.split()
for i in n:
    vector=vec.transform([i])
    pred=clsf.predict(vector)
    print(i,encoder.inverse_transform(pred))

Malaria ['S-Disease']
is ['O']
an ['O']
acute ['O']
febrile ['O']
illness ['O']
caused ['O']
by ['O']
Plasmodium ['B-Organism']
parasites, ['O']
which ['O']
are ['O']
spread ['O']
to ['O']
people ['O']
through ['O']
the ['O']
bites ['O']
of ['O']
infected ['O']
female ['O']
Anopheles ['B-Organism']
mosquitoes. ['O']
It ['O']
is ['O']
preventable ['O']
and ['O']
curable. ['O']


## NAIVE BAYES+COUNT VECTORIZER

In [238]:
# Create Naive Bayes classifier
c = MultinomialNB()

# Train the classifier on the training data
c.fit(X_train_vectors, y_train)

# Predict the labels of the test data
y_pred = c.predict(X_test_vectors)

In [239]:
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

print("Accuracy:", accuracy)

Accuracy: 0.9313773143033891


In [240]:
new="""Malaria is an acute febrile illness caused by Plasmodium parasites, which are spread to people through the bites of infected female Anopheles mosquitoes. It is preventable and curable."""
n=new.split()
for i in n:
    vector=vec.transform([i])
    pred=clsf.predict(vector)
    print(i,encoder.inverse_transform(pred))